In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install gdown

Import glove from google drive

In [ ]:
!gdown --id 1kqbSRxDRxuvs_O7BbyDngj-kHNnjiKxk

In [5]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.7 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 KB 38.5 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [7]:
import csv
import nltk
import string
import pandas as pd
import numpy as np
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
x_train = open("/kaggle/input/dataset-languagemodel/dataset/train.txt", "r")

In [26]:
x_train = open("/kaggle/input/dataset-edit/edited_train.txt", "r")

In [8]:
x_train = open("/kaggle/input/edited-train-half/edited_train_half.txt", "r")

In [27]:
corpus = x_train.read().lower().split('.')
print(len(corpus))
print(type(corpus))
print(corpus[:2])

45938
<class 'list'>
['many are the hours in which i have pondered upon the story that is setforth in the following pages', ' i trust that my instincts are not awrywhen they prompt me to leave the account, in simplicity, as it washanded to me']


In [28]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index
total_unique_words = len(tokenizer.word_index) + 1 
print(total_unique_words)
print(word_index)

78417
{'the': 1, 'of': 2, 'and': 3, 'in': 4, 'to': 5, 'a': 6, 'was': 7, 'that': 8, 'as': 9, 'is': 10, 'i': 11, 'he': 12, 'for': 13, 'with': 14, 'on': 15, 'it': 16, 'his': 17, 'by': 18, 'at': 19, 'from': 20, 'had': 21, "'s": 22, 'not': 23, 'were': 24, 'this': 25, 'but': 26, 'be': 27, 'an': 28, 'which': 29, 'are': 30, 'my': 31, 'or': 32, 'her': 33, 'you': 34, 'have': 35, 'they': 36, 'who': 37, 'one': 38, 'me': 39, 'all': 40, 'she': 41, 'when': 42, 'their': 43, 'there': 44, 'him': 45, 'so': 46, 'its': 47, 'been': 48, 'first': 49, 'has': 50, 'no': 51, 'then': 52, 'more': 53, 'also': 54, 'if': 55, 'after': 56, 'two': 57, 'into': 58, 'new': 59, 'we': 60, 'now': 61, 'would': 62, 'up': 63, 'out': 64, 'time': 65, 'other': 66, 'them': 67, 'where': 68, 'what': 69, 'such': 70, 'than': 71, 'will': 72, 'some': 73, 'made': 74, 'could': 75, 'through': 76, 'these': 77, 'may': 78, 'thou': 79, 'said': 80, 'can': 81, '–': 82, 'over': 83, '1': 84, 'only': 85, 'well': 86, 'v': 87, 'before': 88, 'about': 89,

In [11]:
for line in corpus:
   seqs = tokenizer.texts_to_sequences([line])[0]
print(seqs)

[]


In [17]:
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
print(token_list[:9])

[]


In [32]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)): 
        n_gram_seqs = token_list[i-1:i+2]
        input_sequences.append(n_gram_seqs)
print(len(input_sequences))
print(input_sequences[:10])

783967
[[95, 30, 1], [30, 1, 708], [1, 708, 4], [708, 4, 29], [4, 29, 11], [29, 11, 35], [11, 35, 19765], [35, 19765, 114], [19765, 114, 1], [114, 1, 487]]


In [33]:
max_seq_length = max([len(x) for x in input_sequences])
input_seqs = np.array(pad_sequences(input_sequences, maxlen=max_seq_length, padding='pre'))
print(max_seq_length)
print(input_seqs[11])

3
[487   8  10]


In [34]:
print(input_seqs[18])

[  11 1786    8]


In [35]:
x_values, labels = input_seqs[:, :-1], input_seqs[:, -1]
y_values = tf.keras.utils.to_categorical(labels, num_classes=total_unique_words)
print(x_values[:3])
print(labels[:3])

[[ 95  30]
 [ 30   1]
 [  1 708]]
[  1 708   4]


In [36]:
print(tokenizer.word_index['good'])
print(tokenizer.word_index['great'])

183
133


In [37]:
path = '/kaggle/working/glove.6B.50d.txt'
embeddings_index = {}
with open(path) as f:
  for line in f:
    values = line.split()
    word = values[0]
    coeffs = np.array(values[1:], dtype='float32')
    embeddings_index[word] = coeffs
print(len(list(embeddings_index.items())))
dict(list(embeddings_index.items())[0:2])

400000


{'the': array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
        -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
         2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
         1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
        -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
        -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
         4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
         7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
        -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
         1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
       dtype=float32),
 ',': array([ 0.013441,  0.23682 , -0.16899 ,  0.40951 ,  0.63812 ,  0.47709 ,
        -0.42852 , -0.55641 , -0.364   , -0.23938 ,  0.13001 , -0.063734,
        -0.39575 , -0.48162 ,  0.23291 ,  0.090201, -0.13324 ,  0.078639,
        -0.4

In [38]:
embeddings_matrix = np.zeros((total_unique_words, 50))
for word, i in word_index.items():
   embedding_vector = embeddings_index.get(word)
   if embedding_vector is not None:
     embeddings_matrix[i] = embedding_vector;

In [20]:
!pip install pydot

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [23]:
!pip install graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 KB 2.3 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [39]:
K.clear_session()
model = tf.keras.Sequential([
tf.keras.layers.Embedding(input_dim = total_unique_words, output_dim=50, weights=[embeddings_matrix], input_length=max_seq_length-1, trainable=False),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
tf.keras.layers.Dropout(0.2), 
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dense(total_unique_words , activation='softmax')])
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# tf.keras.utils.plot_model(model, show_shapes=True)

In [45]:
history = model.fit(x_values, y_values, epochs=1, validation_split=0.2, verbose=1, batch_size=256)
model.save('/kaggle/working/model/model.model')

KeyboardInterrupt: 

In [47]:
from tensorflow import keras
model = keras.models.load_model('/kaggle/working/model/model.model')
history = model.fit(x_values, y_values, epochs=1, validation_split=0.2, verbose=1, batch_size=256)
model.save('/kaggle/working/model/model.model')

2450/2450 [==============================] - 470s 189ms/step - loss: 6.6674 - accuracy: 0.1244 - val_loss: 7.5950 - val_accuracy: 0.1001


INFO:tensorflow:Assets written to: /kaggle/working/model/model.model/assets


INFO:tensorflow:Assets written to: /kaggle/working/model/model.model/assets


In [ ]:
from tensorflow import keras
model = keras.models.load_model('/kaggle/working/model/model.model')

In [ ]:
def prediction(seed_text, next_words): 
  for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_seq_length-1, padding='pre')
  predicted = np.argmax(model.predict(token_list, verbose=1), axis=-1)
  ouput_word = ""
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += ' '+output_word
  print(seed_text)
seed_phrase = "I understand that they could meet"
next_words = len("with us, patronize us and do nothing in the end".split())
prediction(seed_text, next_words)